In [14]:
import os
import pandas as pd
import pm4py
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.conformance.alignments.petri_net import algorithm as align
from pm4py.algo.conformance.tokenreplay import algorithm as tr
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.convert import convert_to_petri_net

# .................................................................Data reprocessing.............................................................

def split_processes():
    file_path = os.path.join('Input/Recorded_Business_Tasks.csv')
    df = pd.read_csv(file_path)
    
    grouped = df.groupby('ProcessId')
    group_list = []
    for name, group in grouped:
        group_list.append(group)
    
    
    os.makedirs('Processes')
    for i in range(0,len(group_list)):
        file_name = 'Processes/'+str(group_list[i].iloc[0]['ProcessId'])+'.csv'
        group_list[i].to_csv(file_name, index=False)

def split_instances():
    for j in range (0,50):
        file_path = os.path.join('./Processes/Process_'+str(j+1)+'.csv')
        print(file_path)
        df = pd.read_csv(file_path)
        pr_id=df.loc[1,'ProcessId']
        grouped = df.groupby('RecordingId')
        group_list = []
        for name, group in grouped:
            group_list.append(group)
            
        for i in range(0,len(group_list)):
            if not os.path.exists('Instances/'+str(pr_id)):
                os.makedirs('Instances/'+str(pr_id))
            file_name = 'Instances/'+str(pr_id)+'/'+str(pr_id)+'_Instance_'+str(i+1)+'.csv'
            group_list[i].to_csv(file_name, index=False)

def find_connectors():
    for j in range (0,50):
        connectors=[]
        conSet=set()
        file_path = os.path.join('./Processes/Process_'+str(j+1)+'.csv')
        print(file_path)
        df = pd.read_csv(file_path)
        process=''
        fileContent=''
        for index, row in df.iterrows():
            #connectors.append({'Connector':str(row['Connectors']),'Process':str(row['ProcessId'])})
            process= str(row['ProcessId'])
            connectors.append(str(row['Connectors']))
        for v in set(connectors):
            if  v and v!= 'nan':
                fileContent+= v+'|'
        print(process,fileContent)
        with open('Instances/connectors/connectors.txt', 'a') as file:
            file.write(process+":"+fileContent+'\n')



def modify_first_step(data_form):
    data_form['StepName'] = data_form.apply(lambda row:'initial_'+ str(row['StepName']) if row['StepId']==0 else row['StepName'], axis=1)


def modify_final_step(data_form):
    grouped = data_form.groupby('RecordingId')
    modified_df= pd.DataFrame()
    for group, df in grouped:
        df['StepName'] = df.apply(lambda row:'final_'+ str(row['StepName']) if (row['StepId']==df.iloc[-1]['StepId']) else row['StepName'], axis=1)
        modified_df = pd.concat([modified_df, df])
    return  modified_df
    
def modify_Step_application_name(data_form):
    
    data_form['StepName'] = data_form.apply(lambda row: str(row['StepName'])+'_in_'+str(row['ApplicationProcessName']) if pd.notna(row['ApplicationProcessName']) else row['StepName'], axis=1)

def fill_missing_application_name(data_form):
    
    data_form['ApplicationProcessName'] = data_form.apply(lambda row: 'Operation_system' if not pd.notna(row['ApplicationProcessName']) else row['ApplicationProcessName'], axis=1)


# .................................................................Log generating.............................................................




def generate_xes_logs_from_csv_files(logs_folder_path):
    files = [f for f in os.listdir(logs_folder_path) if os.path.isfile(os.path.join(logs_folder_path, f))]
    dfs=[]
    for file in files:
        print(file)
        df1 = pd.read_csv(logs_folder_path+file) 
        modify_first_step(df1)
        df1=modify_final_step(df1)
        fill_missing_application_name(df1)
        modify_Step_application_name(df1)
        dfs.append(df1)
    dataframe = pd.concat(dfs,axis=0, ignore_index=True)
    dataframe = dataframe[["RecordingId", "TimeStamp", "StepName"]]
    log_columns = ["case:concept:name", "time:timestamp", "concept:name"]
    dataframe.columns = log_columns
    dataframe["case:concept:name"] = dataframe["case:concept:name"].astype(str)
    dataframe["time:timestamp"] = pd.to_datetime(dataframe["time:timestamp"])
    dataframe["concept:name"] = dataframe["concept:name"].astype(str)
    event_log = pm4py.convert_to_event_log(dataframe)
  
    return event_log



def prepare_log_files(directory_path):
    for folder_name in os.listdir(directory_path):
        if folder_name=='Training logs':
            for connectorFolder in os.listdir(directory_path+folder_name):
                train_folder_path= directory_path+folder_name+'/'+connectorFolder+'/UsedInstances/'
                event_log = generate_xes_logs_from_csv_files(train_folder_path)
                pm4py.write_xes(event_log,directory_path+folder_name+'/'+connectorFolder+'/train_log_'+connectorFolder+'.xes')
                
                    
        elif folder_name=='Test logs':
            for connectorFolder in os.listdir(directory_path+folder_name):
                test_folder_path= directory_path+folder_name+'/'+connectorFolder+'/UsedInstances/'
                event_log = generate_xes_logs_from_csv_files(test_folder_path)
                #print(connector_folder_path[:-14])
                pm4py.write_xes(event_log,directory_path+folder_name+'/'+connectorFolder+'/test_log_'+connectorFolder+'.xes')


# .................................................................Process mining and conformance algorithms.............................................................




def conform_alignment_inductive_miner(petri_log,test_log):
    pt = pm4py.discover_process_tree_inductive(petri_log, 0.4)
    net, initial_marking, final_marking = convert_to_petri_net(pt)
    alignement = align.apply_log(test_log, net, initial_marking, final_marking)
    alignmentResult = pd.DataFrame(alignement)
    averageAlignment = alignmentResult["fitness"].mean()
    return averageAlignment

def conform_replay_inductive_miner(petri_log,test_log):
    net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(train_log)
    tokenReplay = tr.apply(test_log, net, initial_marking, final_marking)
    tokenReplayResult = pd.DataFrame(tokenReplay)
    averageReplayFitness = tokenReplayResult["trace_fitness"].mean()
    return averageReplayFitness

def conform_replay_heuristics_miner(petri_log,test_log):
    net, initial_marking, final_marking = heuristics_miner.apply(train_log,parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.5})
    tokenReplay = tr.apply(test_log, net, initial_marking, final_marking)
    tokenReplayResult = pd.DataFrame(tokenReplay)
    averageReplayFitness = tokenReplayResult["trace_fitness"].mean()
    return averageReplayFitness
    
def conform_alignment_heuristics_miner(petri_log,test_log):
    net, initial_marking, final_marking = heuristics_miner.apply(petri_log,parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.99})
    alignement = align.apply_log(test_log, net, initial_marking, final_marking)
    alignmentResult = pd.DataFrame(alignement)
    averageAlignmentFitness = alignmentResult["fitness"].mean()
    return averageAlignmentFitness

# .................................................................Main.............................................................

#prepare_log_files('./')

training_path= 'Training logs/'
testing_path= 'Test logs/'
for train_connectorFolder in os.listdir(training_path):
    train_log = pm4py.read_xes(training_path+train_connectorFolder+'/train_log_'+train_connectorFolder+'.xes')
    for test_connectorFolder in os.listdir(testing_path):
        test_log =  pm4py.read_xes(testing_path+test_connectorFolder+'/test_log_'+test_connectorFolder+'.xes')
        print('heuristics_token_replay','TrainConnector:',train_connectorFolder,'TestConnector:',test_connectorFolder,'Fitness:',conform_replay_heuristics_miner(train_log,test_log))
        print('heuristics_alignment','TrainConnector:',train_connectorFolder,'TestConnector:',test_connectorFolder,'Fitness:',conform_alignment_heuristics_miner(train_log,test_log))
        print('inductive_token_replay','TrainConnector:',train_connectorFolder,'TestConnector:',test_connectorFolder,'Fitness:',conform_replay_inductive_miner(train_log,test_log))
        print('inductive_alignment','TrainConnector:',train_connectorFolder,'TestConnector:',test_connectorFolder,'Fitness:',conform_alignment_inductive_miner(train_log,test_log))

    



parsing log, completed traces :: 100%|█████████████████████████████████████████████████| 10/10 [00:00<00:00, 182.51it/s]
/home/nawras/.local/lib/python3.8/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/nawras/.local/lib/python3.8/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)

parsing log, completed traces :: 100%|███████████████████████████████████████████████████| 5/5 [00:00<00:00, 191.15it/s]
/home/nawras/.local/lib/python3.8/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning

heuristics_token_replay TrainConnector: PowerBI TestConnector: PowerBI Fitness: 0.8793187911040168



aligning log, completed variants :: 100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 24.59it/s]


heuristics_alignment TrainConnector: PowerBI TestConnector: PowerBI Fitness: 0.9726973684210527



replaying log with TBR, completed variants :: 100%|███████████████████████████████████████| 5/5 [00:00<00:00, 23.97it/s]


...

heuristics_alignment TrainConnector: teams TestConnector: teams Fitness: 0.9440438317426653


replaying log with TBR, completed variants :: 100%|███████████████████████████████████████| 3/3 [00:00<00:00, 52.48it/s]


inductive_token_replay TrainConnector: teams TestConnector: teams Fitness: 0.890857100415924


aligning log, completed variants :: 100%|█████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.31s/it]

inductive_alignment TrainConnector: teams TestConnector: teams Fitness: 0.7313519813519814
